In [1]:
import dolfinx
import ufl
import mpi4py
import numpy as np
import petsc4py

In [2]:
petsc4py.PETSc.ScalarType

numpy.complex128

In [3]:
msh = dolfinx.mesh.create_mesh \
( mpi4py.MPI.COMM_WORLD
, ((0, 1, 2),)
, ((0, 0), (0, 1), (1, 1))
, ufl.Mesh(ufl.Cell('triangle')) )

In [4]:
msh.geometry.x

array([[0., 0., 0.],
       [0., 1., 0.],
       [1., 1., 0.]])

In [5]:
for i in [1, 2]:
    msh.topology.create_connectivity(i, 0)

In [6]:
[msh.topology.connectivity(i, 0) for i in [1, 2]]

[<AdjacencyList> with 3 nodes
   0: [0 1 ]
   1: [0 2 ]
   2: [1 2 ],
 <AdjacencyList> with 1 nodes
   0: [0 1 2 ]]

In [7]:
V = dolfinx.fem.FunctionSpace(msh, ('N1curl', 1))

In [8]:
bcf = dolfinx.fem.Function(V, dtype=np.complex128)

In [9]:
bcf.x.array

array([0.+0.j, 0.+0.j, 0.+0.j])

In [10]:
mt = dolfinx.cpp.mesh.MeshTags_int32 \
( msh, 1, [0], [0] )

In [11]:
[dolfinx.fem.locate_dofs_topological(V, 1, [i]) for i in [1, 2]]

[array([1], dtype=int32), array([0], dtype=int32)]

In [12]:
bc = dolfinx.fem.dirichletbc(bcf, [0, 1])

In [13]:
ds = ufl.Measure('ds', msh, subdomain_data=mt)

In [14]:
u = ufl.TrialFunction(V)

In [15]:
v = ufl.TestFunction(V)

In [16]:
a = ufl.inner(ufl.curl(u), ufl.curl(v))*ufl.dx

In [17]:
exf = dolfinx.fem.Function(V, dtype=np.complex128)

In [18]:
exf.x.array[:] = [0, 0, 1]

In [19]:
n = ufl.FacetNormal(msh)

In [25]:
L = ufl.inner \
( exf[0]*n[1]-exf[1]*n[0]
,   v[0]*n[1]-  v[1]*n[0] )*-1j*ds(0)

In [26]:
#L = ufl.inner(ufl.cross(exf, n)[2], ufl.cross(v, n)[2])*ds(0)

In [27]:
problem = dolfinx.fem.petsc.LinearProblem(a, L, [bc])

In [28]:
sol = problem.solve()

In [29]:
sol.x.array

array([0.+0.j , 0.+0.j , 0.-0.5j])

In [30]:
problem.A.convert('dense').getDenseArray()

array([[1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 2.+0.j]])

In [31]:
problem.b.getArray()

array([0.+0.j, 0.+0.j, 0.-1.j])